In [6]:
username = "alisa"
if username == "alisa":
    import sys
    sys.path.append("/home/alisa/git/AstroDeepLearning/")
from ADL.other.detector.scripts import scan_Planck_Unet, extract_cat_Planck
import pandas as pd
from time import sleep
from tqdm import tqdm
from ADL.model import Unet_model


## Создание каталога скоплений из полученной модели сегментации

In [7]:
def scan_and_detect(name: str, model_path: str, add_lfi: bool, step = 16, model_prms={}):
    lfi_path = None
    if add_lfi:
        lfi_path = f"/home/{username}/Data/Planck/healpix_nside2_lfi/"
    scan_Planck_Unet(model_path, f'/home/{username}/Data/Planck/healpix_nside2/', 
                     f'/home/{username}/Data/Planck/scans/{name}_step{step}/',
                     step=step, device='gpu', lfi_path=lfi_path, model_prms=model_prms)
    extract_cat_Planck(f'/home/{username}/Data/Planck/scans/{name}_step{step}/', 
                      f'/home/{username}/Data/Planck/detected_cats/{name}_step{step}.csv', 0.1)

In [3]:
scan_and_detect("check_detection_not_cluster", 
                "/home/alisa/Models/Planck_Unet/pz-not-cluster_8-8/pz-not-cluster_8-8_ep011.hdf5", add_lfi=False,
               step=64)

Slow scan with step 64


/home/alisa/.local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
100%|██████████| 48/48 [1:58:23<00:00, 147.99s/it]  


In [8]:
def find_epoch(path: str, prm: str ="val_loss", mode="min", oth_prm="loss"):
    history = pd.read_csv(path)
    if mode == "min":
        idx = history[prm].argmin()
    elif mode == "eq":
        idx = abs(history[prm] - history[oth_prm]).argmin()
    epoch = history.epoch[idx]
    return epoch

In [ ]:
sleep((167 + 7) * (140 - 15))

In [ ]:
name = "pz-not-cluster_8-8"
epoch = find_epoch(f"/home/alisa/Models/Planck_Unet/{name}/history.csv", mode="min")
step = 16
print(epoch)
scan_and_detect(f"{name}_ep{epoch:03}", 
                f"/home/alisa/Models/Planck_Unet/{name}/{name}_ep{epoch:03}.hdf5", add_lfi=False, step=step)

In [ ]:
sleep(180 * 140)

In [ ]:
name = "pz-not-cluster-lfi_8-8"
epoch = find_epoch(f"/home/alisa/Models/Planck_Unet/{name}/history.csv", mode="min")
step = 16
print(epoch)
scan_and_detect(f"{name}_ep{epoch:03}", 
                f"/home/alisa/Models/Planck_Unet/{name}/{name}_ep{epoch:03}.hdf5", add_lfi=True, step=step)